In [5]:
import sys
sys.path.append('../')
from datasets import datasets
from models import models_edited
from dataproc import extract_wvs
from dataproc import get_discharge_summaries
from dataproc import concat_and_split
from dataproc import build_vocab
from dataproc import vocab_index_descriptions
from dataproc import word_embeddings
from constants import MIMIC_3_DIR, DATA_DIR

import numpy as np
import pandas as pd

from collections import Counter, defaultdict
import csv
import math
import operator



In [13]:
df_disch = pd.read_csv('%s/disch_full.csv' % MIMIC_3_DIR)

In [14]:
sum(df_disch.duplicated(subset=['HADM_ID', 'TEXT']))

0

In [17]:
new_df = df_disch[df_disch.duplicated(subset=['HADM_ID', 'TEXT'])]

In [18]:
len(new_df)

0

In [19]:
new_df = df_disch[df_disch.duplicated(subset=['HADM_ID'])]

In [21]:
print(new_df.head())

    SUBJECT_ID  HADM_ID  CHARTTIME  \
20        9805   177212        NaN   
34         710   114242        NaN   
54        4127   167565        NaN   
59        5239   129387        NaN   
61        5239   125055        NaN   

                                                 TEXT  
20  admission date discharge date date of birth se...  
34  admission date discharge date date of birth se...  
54  admission date discharge date date of birth se...  
59  admission date discharge date service ccu plea...  
61  admission date discharge date service ccu adde...  


In [22]:
len(new_df)

6754

In [23]:
len(df_disch)

59480

In [24]:
new_df_disch = df_disch.groupby(['SUBJECT_ID', 'HADM_ID'])['TEXT'].apply(' '.join).reset_index()

In [25]:
len(new_df_disch)

52726

In [26]:
df_dup = df_disch[df_disch['HADM_ID'] == 188822]

In [27]:
for index, row in df_dup.iterrows():
    print (row['TEXT'])

admission date discharge date date of birth sex m service psychiatry allergies bactrim attending first lf chief complaint i m just so depressed i can t do anything major surgical or invasive procedure none history of present illness pt reports weeks ago he noticed an increase of stress at work and at home at the same time that he was losing weight to gain better control over his diabetes in weeks pt reports having increasingly frequent panic attacks which he controlled with klonopin and occassional xanax he also noticed decrease in sleep energy concentration interest and motivation and an increase in anhedonia over the past weeks his psychiatrist began zoloft which after days made him jittery and this was switched to celexa which he seems to be tolerating better although his neurovegetative sx seem to worsen he denies si but reports fleeting ego dystonic thoughts of hurting his wife and has not acted on any of these thoughts pt reports ruminations concerning whether he is schizophrenic

In [28]:
print(new_df_disch.head())

   SUBJECT_ID  HADM_ID                                               TEXT
0           3   145834  admission date discharge date date of birth se...
1           4   185777  admission date discharge date date of birth se...
2           6   107064  admission date discharge date date of birth se...
3           9   150750  admission date discharge date date of birth se...
4          10   184167  admission date discharge date date of birth se...


In [29]:
df_dup_new = new_df_disch[new_df_disch['HADM_ID'] == 188822]
for index, row in df_dup_new.iterrows():
    print (row['TEXT'])

admission date discharge date date of birth sex m service psychiatry allergies bactrim attending first lf chief complaint i m just so depressed i can t do anything major surgical or invasive procedure none history of present illness pt reports weeks ago he noticed an increase of stress at work and at home at the same time that he was losing weight to gain better control over his diabetes in weeks pt reports having increasingly frequent panic attacks which he controlled with klonopin and occassional xanax he also noticed decrease in sleep energy concentration interest and motivation and an increase in anhedonia over the past weeks his psychiatrist began zoloft which after days made him jittery and this was switched to celexa which he seems to be tolerating better although his neurovegetative sx seem to worsen he denies si but reports fleeting ego dystonic thoughts of hurting his wife and has not acted on any of these thoughts pt reports ruminations concerning whether he is schizophrenic

In [31]:
file = '%s/disch_full.csv' % MIMIC_3_DIR
new_df_disch.to_csv(file, index=False)

In [32]:
df_readmission = pd.read_csv('%s/readmission_labels.csv' % MIMIC_3_DIR)

In [33]:
len(df_readmission['HADM_ID'].unique())

58976

In [34]:
len(new_df_disch['HADM_ID'].unique())

52726

In [35]:
df_merged = new_df_disch.merge(df_readmission, how="left", on="HADM_ID")

In [36]:
df_merged.rename(columns={'test':'READMISSION'}, inplace=True)

In [37]:
len(df_merged)

52726

In [38]:
print(df_merged.head())

   SUBJECT_ID  HADM_ID                                               TEXT  \
0           3   145834  admission date discharge date date of birth se...   
1           4   185777  admission date discharge date date of birth se...   
2           6   107064  admission date discharge date date of birth se...   
3           9   150750  admission date discharge date date of birth se...   
4          10   184167  admission date discharge date date of birth se...   

   READMISSION  
0            0  
1            0  
2            0  
3            0  
4            0  


In [39]:
file = '%s/disch_summary_readmission_labels.csv' % MIMIC_3_DIR
df_merged.to_csv(file, index=False)

In [ ]:
import importlib
importlib.reload(build_vocab)
vocab_min = 3
vname = '%s/vocab.csv' % MIMIC_3_DIR
infile = '%s/disch_summary_readmission_labels.csv' % MIMIC_3_DIR
build_vocab.build_vocab(vocab_min, infile, vname)

reading in data...


In [41]:
w2v_file = word_embeddings.word_embeddings('full', '%s/disch_full.csv' % MIMIC_3_DIR, 100, 0, 5)

building word2vec vocab on ../mimicdata/mimic3/disch_full.csv...
training...
writing embeddings to ../mimicdata/mimic3/processed_full.w2v


In [42]:
Y = 'full'
extract_wvs.gensim_to_embeddings('%s/processed_full.w2v' % MIMIC_3_DIR, '%s/vocab.csv' % MIMIC_3_DIR, Y)

100%|██████████| 49638/49638 [01:35<00:00, 520.14it/s] 


In [63]:
#Creating splits
df_disch = pd.read_csv('%s/disch_summary_readmission_labels.csv' % MIMIC_3_DIR)
msk = np.random.rand(len(df_disch)) < 0.8
train = df_disch[msk]
test = df_disch[~msk]


   SUBJECT_ID  HADM_ID                                               TEXT  \
0           3   145834  admission date discharge date date of birth se...   
2           6   107064  admission date discharge date date of birth se...   
3           9   150750  admission date discharge date date of birth se...   
4          10   184167  admission date discharge date date of birth se...   
5          11   194540  admission date discharge date date of birth se...   

   READMISSION  
0            0  
2            0  
3            0  
4            0  
5            0  
    SUBJECT_ID  HADM_ID                                               TEXT  \
1            4   185777  admission date discharge date date of birth se...   
12          20   157681  admission date discharge date date of birth se...   
18          24   161859  admission date discharge date date of birth se...   
31          37   188670  admission date discharge date date of birth se...   
44          56   181711  admission date disch

In [64]:
file = '%s/disch_sum_train.csv' % MIMIC_3_DIR
train.to_csv(file, index=False)
file = file = '%s/disch_sum_test.csv' % MIMIC_3_DIR
test.to_csv(file, index = False)

In [7]:
import importlib
importlib.reload(get_discharge_summaries)
disch_full_file = get_discharge_summaries.write_discharge_summaries(out_file="%s/disch_full.csv" % MIMIC_3_DIR)

112it [00:00, 1118.95it/s]

processing notes file
writing to ../mimicdata/mimic3/disch_full.csv


2083180it [01:27, 23676.62it/s]
